# Evaluation Notebook (Test Split)

Notebook ringkas untuk evaluasi model pada `test_leadXX.csv`.

Anda hanya perlu mengubah path dataset dan path weights.

## 0. (Opsional) Install dependency

In [ ]:
!pip -q install -r requirements.txt || true

## 1. Konfigurasi path

In [ ]:
import os, sys, json
from pathlib import Path

REPO_DIR = Path('.').resolve()
sys.path.insert(0, str(REPO_DIR))

# ====== UBAH SESUAI LOKASI DATASET ANDA ======
BASE_DIR = '/kaggle/input/NAMA_DATASET/NPZ'
MANIFEST_DIR = '/kaggle/input/NAMA_DATASET/manifests'
OUT_DIR = '/kaggle/working/run_h60_BTDRadar'
os.makedirs(OUT_DIR, exist_ok=True)

# ====== KONFIGURASI EVAL ======
MODE = 'BTDRadar'
HORIZON = 60
BATCH_SIZE = 4
WINDOW_STEPS = 12
CADENCE_MIN = 10
EXPECTED_HW = (128, 128)

# weights: gunakan best.weights.h5 atau last.weights.h5
WEIGHTS = os.path.join(OUT_DIR, 'best.weights.h5')

print('WEIGHTS:', WEIGHTS)


## 2. Import modul + siapkan test generator

In [ ]:
import pandas as pd
import tensorflow as tf

from src.data.npz_sequence_tf import NPZSequence, NormStats
from src.modeling import build_model, compile_model
from src.metrics_tf import F1ScorePerChannel, ThreatScorePerChannel, BiasMetricPerChannel

test_csv = os.path.join(MANIFEST_DIR, f'test_lead{HORIZON}.csv')

# Load norm jika tersedia
norm = None
norm_path = os.path.join(OUT_DIR, 'norm_stats.json')
if os.path.exists(norm_path):
    norm = NormStats.from_json(json.load(open(norm_path, 'r')))

test_seq = NPZSequence(
    base_dir=BASE_DIR,
    manifest_csv=test_csv,
    mode=MODE,
    horizon_min=HORIZON,
    batch_size=BATCH_SIZE,
    window_steps=WINDOW_STEPS,
    cadence_min=CADENCE_MIN,
    shuffle=False,
    expected_hw=EXPECTED_HW,
    flatten_time_channels=True,
    norm_stats=norm,
)

print('test batches:', len(test_seq))


## 3. Build model, compile, load weights

In [ ]:
alpha = None
alpha_path = os.path.join(OUT_DIR, 'alpha.json')
if os.path.exists(alpha_path):
    alpha = json.load(open(alpha_path, 'r')).get('alpha')
print('alpha:', alpha)

H, W = EXPECTED_HW
input_channels = WINDOW_STEPS * test_seq.c_base

model = build_model(input_size=(H, W, input_channels), t_steps=WINDOW_STEPS, with_deep_supervision=True)
model, _ = compile_model(
    model,
    y_train=tf.constant([0.0]),
    steps_per_epoch=1,
    total_epochs=1,
    alpha_override=alpha,
    extra_metrics=[
        F1ScorePerChannel(0, name='f1_T1'),
        F1ScorePerChannel(1, name='f1_T2'),
        F1ScorePerChannel(2, name='f1_T3'),
        ThreatScorePerChannel(0, name='ts_T1'),
        ThreatScorePerChannel(1, name='ts_T2'),
        ThreatScorePerChannel(2, name='ts_T3'),
        BiasMetricPerChannel(0, name='bias_T1'),
        BiasMetricPerChannel(1, name='bias_T2'),
        BiasMetricPerChannel(2, name='bias_T3'),
    ],
)

model.load_weights(WEIGHTS)
print('Loaded weights OK')


## 4. Evaluate + simpan hasil ke JSON/CSV

In [ ]:
results = model.evaluate(test_seq, verbose=1, return_dict=True)
print(results)

out_json = os.path.join(OUT_DIR, f'eval_test_lead{HORIZON}.json')
out_csv  = os.path.join(OUT_DIR, f'eval_test_lead{HORIZON}.csv')
json.dump(results, open(out_json, 'w'), indent=2)
pd.DataFrame([results]).to_csv(out_csv, index=False)
print('Saved:', out_json)
print('Saved:', out_csv)


## 5. (Opsional) Plot learning curve dari train_log.csv

In [ ]:
import matplotlib.pyplot as plt

log_path = os.path.join(OUT_DIR, 'train_log.csv')
if os.path.exists(log_path):
    df = pd.read_csv(log_path)
    cols = [c for c in df.columns if ('pr_auc_T3' in c) or ('loss' in c)]
    print('available cols:', cols)
    if 'val_main_pr_auc_T3' in df.columns:
        plt.figure()
        plt.plot(df['val_main_pr_auc_T3'])
        plt.title('val_main_pr_auc_T3 per epoch')
        plt.xlabel('epoch')
        plt.ylabel('PR-AUC')
        plt.show()
else:
    print('train_log.csv not found, skip plot')
